In [ ]:
from influence_info.influencers.experiments.pairwise_applicator import *

dataset = "IMDB"

import os
os.makedirs(f"figures/{dataset}", exist_ok=True)

In [ ]:
values, training_idx, validation_idx = get_all_values(
    f"outputs/{dataset}/basic/transformer_text_classifier",
    (["influence_function/*"] if "full" not in dataset else []) +
    [
        # "tracein_softmax",
        "influence_function_softmax/*",
        "representer_points_with_sec",
        "nearest_neighbors/*",
        # "nearest_neighbors_sep/*",
    ],
)

In [ ]:
values[5][2].shape

In [ ]:
import json

predictions = [
    json.loads(line)
    for line in open(f"outputs/{dataset}/basic/transformer_text_classifier/predictions.test.jsonl")
]
predictions = {x["idx"]: x for x in predictions}
validation_predicted_labels = [predictions[x]["predicted_labels"] for x in validation_idx]

predictions = [
    json.loads(line)
    for line in open(f"outputs/{dataset}/basic/transformer_text_classifier/predictions.train.jsonl")
]
predictions = {x["idx"]: x for x in predictions}
training_predicted_labels = [predictions[x]["predicted_labels"] for x in training_idx]


In [ ]:
import numpy as np

for i, val in enumerate(values):
    if not (
        val[0] == "nearest_neighbors" or val[0] == "influence_function" or val[0] == "nearest_neighbors_sep"
    ):
        values[i] = (val[0], val[1], val[2][np.arange(val[2].shape[0]), :, validation_predicted_labels])
        print(values[i][2].shape)


In [ ]:
from scipy.stats import spearmanr

applicator = mean_pairwise_applicator(lambda x, y: spearmanr(x.flatten(), y.flatten()).correlation)

top_n_func = lambda x, y, n: len(set(np.argsort(x)[-n:]) & set(np.argsort(y)[-n:])) / float(n)
top_n_applicator = lambda n: pairwise_applicator(lambda x, y: top_n_func(x, y, n), np.array)


In [ ]:
vals = [5, 10, 20, 30, 50, 80, 100, 200, 400, 500] if "full" not in dataset else [10, 50]
ys = []
for n in vals:
    triplets = pairwise_experiment(values, top_n_applicator(n))
    ys.append(triplets[2][2])
    if n == 10:
        top10 = triplets
    if n == 50:
        top50 = triplets

In [ ]:
ys = np.array(ys)
ys = pd.DataFrame(ys)
ys.index = vals
ys = ys.melt(ignore_index=False).reset_index()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

if "full" not in dataset :
    ax = sns.lineplot(data=ys, x="index", y="value")
    ax.set_xlabel("N")
    ax.set_ylabel("proportion of common elements")
    ax.get_figure().savefig(f"figures/{dataset}/top_N.pdf", bbox_inches="tight")


In [ ]:
f = plt.figure(figsize=(8, 5))
ax = plot_pairwise_matrix([(x, y, np.mean(v)) for x, y, v in top10])
plt.show()
ax.get_figure().savefig(f"figures/{dataset}/top_10.pdf", bbox_inches="tight")

f = plt.figure(figsize=(8, 5))
ax = plot_pairwise_matrix([(x, y, np.mean(v)) for x, y, v in top50])
plt.show()
ax.get_figure().savefig(f"figures/{dataset}/top_50.pdf", bbox_inches="tight")


In [ ]:
triplets = pairwise_experiment(values, applicator)
f = plt.figure(figsize=(8, 5))
ax = plot_pairwise_matrix(triplets)
ax.get_figure().savefig(f"figures/{dataset}/corr.pdf", bbox_inches="tight")

In [ ]:
if "full" not in dataset :
    no_sent_bert_triplets = removefilter(
        triplets,
        [
            ('nearest_neighbors_sep', 'sim_func:cos'),
            ('nearest_neighbors_sep', 'sim_func:dot'),
            ('nearest_neighbors_sep', 'sim_func:euc')
        ],
    )
    ax = plot_pairwise_matrix(no_sent_bert_triplets)
    ax.get_figure().savefig(f"figures/{dataset}/corr_wo_sentbert.pdf", bbox_inches="tight")

In [ ]:
predictions["train_0"]["logits"]